## PySAL Change Log Statistics: Table Generation

This notebook generates the summary statistics for use in the 6-month releases of PySAL, which is now a meta package. 

It assumes the subpackages have been git cloned in a directory below the location of this notebook. It also requires network connectivity for some of the reporting.

Run this notebook after `100-gitcount.ipynb`


In [24]:
from __future__ import print_function
import os
import json
import re
import sys
import pandas
import subprocess
from subprocess import check_output

#import yaml
from datetime import datetime, timedelta, time

from dateutil.parser import parse
import pytz

utc=pytz.UTC

try:
    from urllib import urlopen
except:
    from urllib.request import urlopen

from release_info import release_date, start_date, PYSALVER

since = datetime.combine(start_date, time(0,0))


In [25]:
release_date

datetime.date(2022, 7, 23)

In [26]:
CWD = os.path.abspath(os.path.curdir)

In [27]:
CWD

'/home/jovyan/work/tools'

In [28]:
with open('frozen.txt', 'r') as package_list:
    packages = package_list.readlines()
    packages = dict([package.strip().split(">=") for package in packages])

In [29]:
packages

{'libpysal': '4.6.2',
 'access': '1.1.8',
 'esda': '2.4.1',
 'giddy': '2.3.3',
 'inequality': '1.0.0',
 'pointpats': '2.2.0',
 'segregation': '2.3.1',
 'spaghetti': '1.6.6',
 'mgwr': '2.1.2',
 'momepy': '0.5.3',
 'spglm': '1.0.8',
 'spint': '1.0.7',
 'spreg': '1.2.4',
 'spvcm': '0.3.0',
 'tobler': '0.8.2',
 'mapclassify': '2.4.3',
 'splot': '1.1.5.post1',
 'spopt': '0.4.1'}

import pysal
packages['pysal'] = pysal.__version__

In [30]:
import pickle

In [31]:
issues_closed = pickle.load(open("issues_closed.p", 'rb'))
pulls_closed = pickle.load(open('pulls_closed.p', 'rb'))

In [32]:
type(issues_closed)

dict

In [33]:
issues_closed.keys()

dict_keys(['libpysal', 'access', 'esda', 'giddy', 'inequality', 'pointpats', 'segregation', 'spaghetti', 'mgwr', 'momepy', 'spglm', 'spint', 'spreg', 'spvcm', 'tobler', 'mapclassify', 'splot', 'spopt', 'pysal'])

In [34]:
from release_info import get_pypi_info, get_github_info, clone_masters

In [35]:
#github_releases = get_github_info()

github_releases = pickle.load(open("releases.p", 'rb'))


In [36]:
from datetime import datetime

In [37]:
#pysal_date = datetime.strptime('2021-07-31T12:00:00Z', '%Y-%m-%dT%H:%M:%SZ')
pysal_date = release_date
#ISO8601 = "%Y-%m-%dT%H:%M:%SZ"


In [38]:
pysal_rel = {'version': f'v{PYSALVER}',
            'release_date': release_date}
github_releases['pysal'] = pysal_rel

In [39]:
github_releases

{'libpysal': {'version': 'v4.6.2',
  'url': 'https://api.github.com/repos/pysal/libpysal/tarball/v4.6.2',
  'release_date': datetime.datetime(2022, 3, 3, 17, 30, 4)},
 'access': {'version': 'v1.1.8',
  'url': 'https://api.github.com/repos/pysal/access/tarball/v1.1.8',
  'release_date': datetime.datetime(2022, 6, 4, 20, 15, 41)},
 'esda': {'version': 'v2.4.1',
  'url': 'https://api.github.com/repos/pysal/esda/tarball/v2.4.1',
  'release_date': datetime.datetime(2021, 7, 27, 12, 54, 27)},
 'giddy': {'version': 'v2.3.3',
  'url': 'https://api.github.com/repos/pysal/giddy/tarball/v2.3.3',
  'release_date': datetime.datetime(2020, 6, 10, 4, 59, 45)},
 'inequality': {'version': 'v1.0.0',
  'url': 'https://api.github.com/repos/pysal/inequality/tarball/v1.0.0',
  'release_date': datetime.datetime(2018, 10, 31, 22, 28, 18)},
 'pointpats': {'version': 'v2.2.0',
  'url': 'https://api.github.com/repos/pysal/pointpats/tarball/v2.2.0',
  'release_date': datetime.datetime(2020, 7, 27, 22, 17, 33)},
 

In [41]:
from datetime import datetime
datetime.fromtimestamp(0)
ISO8601 = "%Y-%m-%dT%H:%M:%SZ"


final_pulls = {}
final_issues = {}
for package in packages:
    filtered_issues = []
    filtered_pulls = []
    released = github_releases[package]['release_date']
    package_pulls = pulls_closed[package]
    package_issues = issues_closed[package]
    for issue in package_issues:
        #print(issue['number'], issue['title'], issue['closed_at'])
        closed = datetime.strptime(issue['closed_at'], ISO8601)
        if closed <= released and closed > since:
            filtered_issues.append(issue)
    final_issues[package] = filtered_issues
    for pull in package_pulls:
        #print(pull['number'], pull['title'], pull['closed_at'])
        closed = datetime.strptime(pull['closed_at'], ISO8601)
        if closed <= released and closed > since:
            filtered_pulls.append(pull)
    final_pulls[package] = filtered_pulls
    print(package, released, len(package_issues), len(filtered_issues), len(package_pulls),
         len(filtered_pulls))

libpysal 2022-03-03 17:30:04 14 7 0 0
access 2022-06-04 20:15:41 14 9 7 5
esda 2021-07-27 12:54:27 6 0 0 0
giddy 2020-06-10 04:59:45 2 0 0 0
inequality 2018-10-31 22:28:18 1 0 0 0
pointpats 2020-07-27 22:17:33 0 0 0 0
segregation 2022-06-18 19:58:17 10 10 0 0
spaghetti 2022-07-23 02:32:44 13 13 0 0
mgwr 2020-09-08 21:20:34 1 0 1 0
momepy 2022-04-09 13:31:49 17 9 0 0
spglm 2020-09-08 20:34:08 0 0 0 0
spint 2020-09-09 02:28:50 0 0 0 0
spreg 2021-06-29 19:21:48 10 0 7 0
spvcm 2020-02-02 19:42:39 0 0 0 0
tobler 2021-06-30 18:24:55 13 0 3 0
mapclassify 2021-07-27 03:06:32 9 0 4 0
splot 2022-04-13 21:13:27 14 11 10 9
spopt 2022-06-13 15:07:03 42 35 0 0


In [42]:
issue_details = final_issues
pull_details = final_pulls

In [43]:
packages

{'libpysal': '4.6.2',
 'access': '1.1.8',
 'esda': '2.4.1',
 'giddy': '2.3.3',
 'inequality': '1.0.0',
 'pointpats': '2.2.0',
 'segregation': '2.3.1',
 'spaghetti': '1.6.6',
 'mgwr': '2.1.2',
 'momepy': '0.5.3',
 'spglm': '1.0.8',
 'spint': '1.0.7',
 'spreg': '1.2.4',
 'spvcm': '0.3.0',
 'tobler': '0.8.2',
 'mapclassify': '2.4.3',
 'splot': '1.1.5.post1',
 'spopt': '0.4.1'}

In [44]:
github_releases['pysal']['release_date'] = release_date

In [45]:
released

datetime.datetime(2022, 6, 13, 15, 7, 3)

In [46]:
packages.keys()

dict_keys(['libpysal', 'access', 'esda', 'giddy', 'inequality', 'pointpats', 'segregation', 'spaghetti', 'mgwr', 'momepy', 'spglm', 'spint', 'spreg', 'spvcm', 'tobler', 'mapclassify', 'splot', 'spopt'])

In [47]:
spvcm = packages['spvcm']

In [60]:
## skip packages not released since last meta release

# handle meta
mrd = github_releases['pysal']['release_date']
github_releases['pysal']['release_date'] =  datetime.combine(mrd, time(0,0))


In [61]:
for package in github_releases:
    if github_releases[package]['release_date']>since:
        print("new: ",package)
    else:
        print('old:', package)

new:  libpysal
new:  access
old: esda
old: giddy
old: inequality
old: pointpats
new:  segregation
new:  spaghetti
old: mgwr
new:  momepy
old: spglm
old: spint
old: spreg
old: spvcm
old: tobler
old: mapclassify
new:  splot
new:  spopt
new:  pysal


In [59]:
github_releases[package]['release_date']

datetime.date(2022, 7, 23)

In [55]:
since

datetime.datetime(2022, 2, 1, 0, 0)

In [51]:
github_releases

{'libpysal': {'version': 'v4.6.2',
  'url': 'https://api.github.com/repos/pysal/libpysal/tarball/v4.6.2',
  'release_date': datetime.datetime(2022, 3, 3, 17, 30, 4)},
 'access': {'version': 'v1.1.8',
  'url': 'https://api.github.com/repos/pysal/access/tarball/v1.1.8',
  'release_date': datetime.datetime(2022, 6, 4, 20, 15, 41)},
 'esda': {'version': 'v2.4.1',
  'url': 'https://api.github.com/repos/pysal/esda/tarball/v2.4.1',
  'release_date': datetime.datetime(2021, 7, 27, 12, 54, 27)},
 'giddy': {'version': 'v2.3.3',
  'url': 'https://api.github.com/repos/pysal/giddy/tarball/v2.3.3',
  'release_date': datetime.datetime(2020, 6, 10, 4, 59, 45)},
 'inequality': {'version': 'v1.0.0',
  'url': 'https://api.github.com/repos/pysal/inequality/tarball/v1.0.0',
  'release_date': datetime.datetime(2018, 10, 31, 22, 28, 18)},
 'pointpats': {'version': 'v2.2.0',
  'url': 'https://api.github.com/repos/pysal/pointpats/tarball/v2.2.0',
  'release_date': datetime.datetime(2020, 7, 27, 22, 17, 33)},
 

In [ ]:
since_date = '--since="{start}"'.format(start=start_date.strftime("%Y-%m-%d"))
since_date
                                        

In [ ]:
# commits
cmd = ['git', 'log', '--oneline', since_date]

activity = {}
total_commits = 0
tag_dates = {}
ncommits_total = 0
for subpackage in packages:
    released = github_releases[subpackage]['release_date']
    tag_date = released.strftime("%Y-%m-%d")
    tag_dates[subpackage] = tag_date
    print(tag_date)
    #tag_date = tag_dates[subpackage]
    ncommits = 0
    if released > since:
        os.chdir(CWD)
        os.chdir('tmp/{subpackage}'.format(subpackage=subpackage))
        cmd_until = cmd + ['--until="{tag_date}"'.format(tag_date=tag_date)]
        ncommits = len(check_output(cmd_until).splitlines())
        ncommits_total = len(check_output(cmd).splitlines())
    print(subpackage, ncommits_total, ncommits, tag_date)
    total_commits += ncommits
    activity[subpackage] = ncommits

In [ ]:
activity

In [ ]:
subpackage

In [ ]:
CWD

In [ ]:
# commits
cmd = ['git', 'log', '--oneline', since_date]

activity = {}
total_commits = 0
for subpackage in packages:
    ncommits = 0
    tag_date = tag_dates[subpackage]
    released = github_releases[subpackage]['release_date']
    if released > since:
        os.chdir(CWD)
        os.chdir('tmp/{subpackage}'.format(subpackage=subpackage))
        cmd_until = cmd + ['--until="{tag_date}"'.format(tag_date=tag_date)]
        ncommits = len(check_output(cmd_until).splitlines())
        print(ncommits)
        ncommits_total = len(check_output(cmd).splitlines())
        print(subpackage, ncommits_total, ncommits, tag_date)
    total_commits += ncommits
    activity[subpackage] = ncommits

In [ ]:
identities = {'Levi John Wolf': ('ljwolf', 'Levi John Wolf'),
              'Serge Rey': ('Serge Rey', 'Sergio Rey', 'sjsrey', 'serge'),
              'Wei Kang': ('Wei Kang', 'weikang9009'),
              'Dani Arribas-Bel': ('Dani Arribas-Bel', 'darribas'),
              'Antti Härkönen': ( 'antth', 'Antti Härkönen', 'Antti Härkönen', 'Antth'  ),
              'Juan C Duque': ('Juan C Duque', "Juan Duque"),
              'Renan Xavier Cortes': ('Renan Xavier Cortes', 'renanxcortes', 'Renan Xavier Cortes'   ),
              'Taylor Oshan': ('Tayloroshan', 'Taylor Oshan', 'TaylorOshan'),
              'Tom Gertin': ('@Tomgertin', 'Tom Gertin', '@tomgertin')
}

def regularize_identity(string):
    string = string.decode()
    for name, aliases in identities.items():
        for alias in aliases:
            if alias in string:
                string = string.replace(alias, name)
    if len(string.split(' '))>1:
        string = string.title()
    return string.lstrip('* ')

In [ ]:
author_cmd = ['git', 'log', '--format=* %aN', since_date]

In [ ]:
author_cmd.append('blank')

In [ ]:
author_cmd

In [ ]:
from collections import Counter

In [ ]:
tag_dates

In [ ]:
authors_global = set()
authors = {}
global_counter = Counter()
counters = dict()
cmd = ['git', 'log', '--oneline', since_date]
total_commits = 0
activity = {}
for subpackage in packages:
    ncommits = 0
    released = github_releases[subpackage]['release_date']
    if released > since:
        os.chdir(CWD)
        os.chdir('tmp/{subpackage}'.format(subpackage=subpackage))
        ncommits = len(check_output(cmd).splitlines())
        print(cmd)
        tag_date = tag_dates[subpackage]
        tag_date = (datetime.strptime(tag_date, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d')
        author_cmd[-1] = '--until="{tag_date}"'.format(tag_date=tag_date)
        #cmd_until = cmd + ['--until="{tag_date}"'.format(tag_date=tag_date)]
        print(subpackage, author_cmd)


        all_authors = check_output(author_cmd).splitlines()
        counter = Counter([regularize_identity(author) for author in all_authors])
        global_counter += counter
        counters.update({subpackage: counter})
        unique_authors = sorted(set(all_authors))
        authors[subpackage] =  unique_authors
        authors_global.update(unique_authors)
    total_commits += ncommits
    activity[subpackage] = ncommits

In [ ]:
author_cmd

In [ ]:
subpackage

In [ ]:
authors_global

In [ ]:
activity

In [ ]:
def get_tag(title, level="##", as_string=True):
    words = title.split()
    tag = "-".join([word.lower() for word in words])
    heading = level+" "+title
    line = "\n\n<a name=\"{}\"></a>".format(tag)
    lines = [line]
    lines.append(heading)
    if as_string:
        return "\n".join(lines)
    else:
        return lines

In [ ]:
subs = issue_details.keys()
table = []
txt = []
lines = get_tag("Changes by Package", as_string=False)

for sub in subs:
    total= issue_details[sub]
    pr = pull_details[sub]
    
    row = [sub, activity[sub], len(total), len(pr)]
    table.append(row)
    #line = "\n<a name=\"{sub}\"></a>".format(sub=sub)
    #lines.append(line)
    #line = "### {sub}".format(sub=sub)
    #lines.append(line)
    lines.extend(get_tag(sub.lower(), "###", as_string=False))
    for issue in total:
        url = issue['html_url']
        title = issue['title']
        number = issue['number']
        line = "* [#{number}:]({url}) {title} ".format(title=title,
                                                     number=number,
                                                     url=url)
        lines.append(line)



In [ ]:
sub

In [ ]:
os.chdir(CWD)

import pandas

In [ ]:
df = pandas.DataFrame(table, columns=['package', 'commits', 'total issues', 'pulls'])

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
type(counters)

In [ ]:
df.sort_values(['commits','pulls'], ascending=False)\
  .to_html('./commit_table.html', index=None)

In [ ]:
df.sum()

In [ ]:
contributor_table = pandas.DataFrame.from_dict(counters).fillna(0).astype(int).T

In [ ]:
contributor_table.to_html('./contributor_table.html')

In [ ]:
totals = contributor_table.sum(axis=0).T
totals.sort_index().to_frame('commits')

In [ ]:
totals = contributor_table.sum(axis=0).T
totals.sort_index().to_frame('commits').to_html('./commits_by_person.html')

In [ ]:
totals

In [ ]:
n_commits = df.commits.sum()
n_issues = df['total issues'].sum()
n_pulls = df.pulls.sum()

In [ ]:
n_commits

In [ ]:
#Overall, there were 719 commits that closed 240 issues, together with 105 pull requests across 12 packages since our last release on 2017-11-03.
#('{0} Here is a really long '
#           'sentence with {1}').format(3, 5))
line = ('Overall, there were {n_commits} commits that closed {n_issues} issues,'  
    ' together with {n_pulls} pull requests since our last release' 
        ' on {since_date}.\n'.format(n_commits=n_commits, n_issues=n_issues,
        n_pulls=n_pulls, since_date = start_date))

In [ ]:
line

## append html files to end of changes.md with tags for toc

In [ ]:
with open('changes.md', 'w') as fout:
    fout.write(line)
    fout.write("\n".join(lines))
    fout.write(get_tag("Contributors"))
    fout.write("\n\nMany thanks to all of the following individuals who contributed to this release:\n\n")
    
    
    
    totals = contributor_table.sum(axis=0).T
    contributors = totals.index.values
    contributors.sort()
    contributors = contributors.tolist() 
    contributors = [ f'\n - {contributor}' for contributor in contributors]
    fout.write("".join(contributors))
    
